In [ ]:
import urllib.parse
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

params=urllib.parse.quote_plus('Driver={ODBC Driver 17 for SQL Server};Server=LAPTOP-LDP5RKBS;Database=News;Trusted_Connection=yes;')

app=Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI']=f'mssql+pyodbc:///?odbc_connect={params}'
db=SQLAlchemy(app)

class User(db.Model):
    __tablename__='stories'
    id=db.Column(db.INTEGER,primary_key=True)
    category = db.Column(db.String('max'),unique=False,nullable=False)
    subcategory = db.Column(db.String('max'),unique=False,nullable=False)
    title=db.Column(db.String('max'),unique=False,nullable=False)
    date=db.Column(db.String('max'),unique=False,nullable=False)
    abstract=db.Column(db.String('max'),unique=False,nullable=False)
    contents=db.Column(db.String('max'),unique=False,nullable=False)
    url=db.Column(db.String('max'),unique=False,nullable=False)
    

    
    def __init__(self,category,subcategory,title,date,abstract,contents,url):
        self.category=category
        self.subcategory=subcategory
        self.title=title
        self.date=date
        self.abstract=abstract
        self.contents=contents
        self.url=url

class User(db.Model):
    __tablename__='videos'
    id=db.Column(db.INTEGER,primary_key=True)
    title=db.Column(db.String('max'),unique=False,nullable=False)
    date=db.Column(db.String('max'),unique=False,nullable=False)
    abstract=db.Column(db.String('max'),unique=False,nullable=False)
    contents=db.Column(db.String('max'),unique=False,nullable=False)
    url=db.Column(db.String('max'),unique=False,nullable=False)
    
    
    def __init__(self,title,date,abstract,contents,url):
        self.title=title
        self.date=date
        self.abstract=abstract
        self.contents=contents
        self.url=url

if __name__=='__main__':
    with app.app_context():
        db.drop_all()
        db.create_all()      

In [4]:
import urllib.parse
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# 初始化 Flask 應用程式與資料庫
def create_app(SQLSERVER, SQL_DATABASE, SQL_USERNAME,SQL_PASSWORD):
    app = Flask(__name__)
    params = urllib.parse.quote_plus(
        f'Driver={{ODBC Driver 17 for SQL Server}};Server={SQLSERVER};Database={SQL_DATABASE};UID={SQL_USERNAME};PWD={SQL_PASSWORD};Trusted_Connection=yes;'
    )
    app.config['SQLALCHEMY_DATABASE_URI'] = f'mssql+pyodbc:///?odbc_connect={params}'
    return app

# 初始化資料庫
def init_db(app):
    db = SQLAlchemy(app)
    return db

# 定義 Stories Model
def define_stories_model(db):
    class Stories(db.Model):
        __tablename__ = 'stories'
        id = db.Column(db.INTEGER, primary_key=True)
        category = db.Column(db.String('max'), unique=False, nullable=False)
        subcategory = db.Column(db.String('max'), unique=False, nullable=False)
        title = db.Column(db.String('max'), unique=False, nullable=False)
        date = db.Column(db.String('max'), unique=False, nullable=False)
        abstract = db.Column(db.String('max'), unique=False, nullable=False)
        contents = db.Column(db.String('max'), unique=False, nullable=False)
        url = db.Column(db.String('max'), unique=False, nullable=False)

        def __init__(self, category, subcategory, title, date, abstract, contents, url):
            self.category = category
            self.subcategory = subcategory
            self.title = title
            self.date = date
            self.abstract = abstract
            self.contents = contents
            self.url = url

    return Stories

# 建立資料庫
def setup_database(app, db):
    with app.app_context():
        db.drop_all()
        db.create_all()

if __name__ == '__main__':
    app = create_app("SQLSERVER", "SQL_DATABASE", "SQL_USERNAME","SQL_PASSWORD")
    db = init_db(app)
    
    Stories = define_stories_model(db)
    
    setup_database(app, db)


In [ ]:
#文章
import pyodbc
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

def setup_database_connection():
    conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                          'Server=SQLSERVER;'
                          'Database=SQL_DATABASE;'
                          'UID=SQL_USERNAME;'
                          'PWD=SQL_PASSWORD;'
                          'Trusted_Connection=yes;')
    return conn

def setup_webdriver(chrome_path):
    service = Service(chrome_path)
    driver = webdriver.Chrome(service=service)
    driver.implicitly_wait(10)
    return driver

def perform_search(driver, url, keyword):
    driver.get(url)
    time.sleep(2)
    driver.find_element(By.ID, 'headerSearchIcon').click()
    driver.find_element(By.CLASS_NAME, 'search-bar__input').send_keys(keyword)
    driver.find_element(By.CLASS_NAME, 'search-bar__submit').click()
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="search"]/div[1]/div[2]/div/div/ul/li[2]/label').click()
    time.sleep(10)

def extract_articles(driver, cur, page_count=1):
    for _ in range(page_count):
        content_div = driver.find_element(By.CLASS_NAME, 'container__field-links')
        divs_in_content = content_div.find_elements(By.XPATH, './div')
        div_count = len(divs_in_content)
        
        for x in range(1, div_count + 1):
            extract_and_save_article(driver, cur, x)
        navigate_to_next_page(driver)

def extract_and_save_article(driver, cur, index):
    titles = driver.find_element(By.XPATH, f'//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div[{index}]/a[2]/div/div[1]/span').text
    publication_dates = driver.find_element(By.XPATH, f'//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div[{index}]/a[2]/div/div[2]').text
    abstracts = driver.find_element(By.XPATH, f'//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div[{index}]/a[2]/div/div[3]').text
    elements = driver.find_element(By.XPATH, f'//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div[{index}]/a[2]')
    urls = elements.get_attribute('href')

    driver.get(urls)
    categories, subcategories = extract_categories(driver)
    content = extract_content(driver)

    cur.execute('''INSERT INTO stories(category,subcategory,title,date,abstract,contents,url) 
                   VALUES(?,?,?,?,?,?,?);''', (categories, subcategories, titles, publication_dates, abstracts, content, urls))
    cur.connection.commit()
    driver.back()
    time.sleep(1)

def extract_categories(driver):
    try:
        categories = driver.find_element(By.CLASS_NAME, 'breadcrumb__parent-link').text
        subcategories = driver.find_element(By.CLASS_NAME, 'breadcrumb__child-link').text
    except:
        categories = " "
        subcategories = " "
    return categories, subcategories

def extract_content(driver):
    try:
        article_content = driver.find_element(By.CLASS_NAME, 'article__content')
        time.sleep(2)
        paragraphs = article_content.find_elements(By.TAG_NAME, 'p')
        time.sleep(2)
        content = "\n".join([p.text for p in paragraphs])
    except:
        content = " "
    return content

def navigate_to_next_page(driver):
    try:
        driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[4]/div/div[3]').click()
        time.sleep(10)
    except:
        print("No more pages.")

def main():
    conn = setup_database_connection()
    cur = conn.cursor()
    cur.fast_executemany = True

    chrome_path = "../chromedriver-win32/chromedriver.exe"

    driver = setup_webdriver(chrome_path)

    url = 'https://edition.cnn.com/'
    keyword = 'Artificial Intelligence'
    
    perform_search(driver, url, keyword)
    extract_articles(driver, cur, page_count=4)  # 控制頁數
    driver.quit()

if __name__ == '__main__':
    main()


In [18]:
#文章類
import pyodbc
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

conn=pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};''Server=LAPTOP-LDP5RKBS;''Database=News;''Trusted_Connection=yes;')
chrome_path = 'C:\\Users\\USER\\Downloads\\chromedriver-win32\\chromedriver-win32\\chromedriver.exe'  
service = Service(chrome_path)
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(10)

url = 'https://edition.cnn.com/'
driver.get(url)
cur = conn.cursor()
cur.fast_executemany = True
time.sleep(2)

driver.find_element(By.ID, 'headerSearchIcon').click() 

keyword = 'Artificial Intelligence'
driver.find_element(By.CLASS_NAME, 'search-bar__input').send_keys(keyword)
driver.find_element(By.CLASS_NAME, 'search-bar__submit').click() 
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="search"]/div[1]/div[2]/div/div/ul/li[2]/label').click() 
time.sleep(10)

content_div = driver.find_element(By.CLASS_NAME, 'container__field-links')
divs_in_content = content_div.find_elements(By.XPATH, './div')
div_count = len(divs_in_content)

#第一頁
for x in range(1, div_count+1):                 
    titles = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[1]/span').text  #標題
    publication_dates =  driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[2]').text   #日期  
    abstracts = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[3]').text    #文章摘要 
    elements = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]')
    urls = elements.get_attribute('href')   #連結
    
    driver.get(urls)    
    try:
        categories = driver.find_element(By.CLASS_NAME, 'breadcrumb__parent-link').text  #類別
        subcategories = driver.find_element(By.CLASS_NAME, 'breadcrumb__child-link').text  #子類別
    except:
        categories = " "
        subcategories = " "
        
    try:
        a = driver.find_element(By.CLASS_NAME, 'article__content')
        time.sleep(2)
        paragraphs = a.find_elements(By.TAG_NAME, 'p')
        time.sleep(2)
        content_str = [p.text for p in paragraphs]
        content = "\n".join(content_str)
    except:
        content = " "
        
    cur.execute('''INSERT INTO stories(category,subcategory,title,date,abstract,contents,url) VALUES(?,?,?,?,?,?,?);''',(categories,subcategories,titles,publication_dates,abstracts,content,urls))
    conn.commit()
    time.sleep(1)

    driver.back()
    time.sleep(1)

for y in range(1,4):               
    pages=driver.find_element(By.XPATH,'//*[@id="search"]/div[2]/div/div[4]/div/div[3]').click()
    time.sleep(10) 
    content_div_ano = driver.find_element(By.CLASS_NAME, 'container__field-links')
    divs_in_content_ano = content_div_ano.find_elements(By.XPATH, './div')
    div_count_ano = len(divs_in_content_ano)
    for z in range(1, div_count_ano+1):                     
        titles = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[1]/span').text  #標題
        publication_dates =  driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[2]').text   #日期  
        abstracts = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[3]').text    #文章摘要 
        elements = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]')
        urls = elements.get_attribute('href')   #連結
        
        driver.get(urls)    
        try:
            categories = driver.find_element(By.CLASS_NAME, 'breadcrumb__parent-link').text  #類別
            subcategories = driver.find_element(By.CLASS_NAME, 'breadcrumb__child-link').text  #子類別
        except:
            categories = " "
            subcategories = " "
        
        try:
            a = driver.find_element(By.CLASS_NAME, 'article__content')
            time.sleep(2)
            paragraphs = a.find_elements(By.TAG_NAME, 'p')
            time.sleep(2)
            content_str = [p.text for p in paragraphs]
            content = "\n".join(content_str)
        except:
            content = " "

        cur.execute('''INSERT INTO stories(category,subcategory,title,date,abstract,contents,url) VALUES(?,?,?,?,?,?,?);''',(categories,subcategories,titles,publication_dates,abstracts,content,urls))
        conn.commit()
        time.sleep(1)

        driver.back()
        time.sleep(1)

driver.quit() 
    

In [ ]:
#影片類
import pyodbc
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

conn=pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};''Server=DESKTOP-4JUNPCV;''Database=News;''Trusted_Connection=yes;')
chrome_path = 'C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver.exe'  
service = Service(chrome_path)
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(10)

url = 'https://edition.cnn.com/'
driver.get(url)
cur = conn.cursor()
time.sleep(2)

driver.find_element(By.ID, 'headerSearchIcon').click() 

keyword = 'Artificial Intelligence'
driver.find_element(By.CLASS_NAME, 'search-bar__input').send_keys(keyword)
driver.find_element(By.CLASS_NAME, 'search-bar__submit').click() 
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="search"]/div[1]/div[2]/div/div/ul/li[3]/label').click() 
time.sleep(10)

#第一頁
for x in range(1,7):                 
    titles = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[1]/span').text  #標題
    publication_dates =  driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[2]').text   #日期  
    abstracts = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]/div/div[3]').text    #文章摘要 
    elements = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(x)+']/a[2]')
    urls = elements.get_attribute('href')   #連結
    
    driver.get(urls)           
    a = driver.find_elements(By.XPATH, '/html/body/div[1]/section[4]/section/div/section/div/div/div[1]/div/div[3]/div[5]')
    time.sleep(2)
    for item in a:
        content = item.text   #內文
    cur.execute('''INSERT INTO videos(title,date,abstract,contents,url) VALUES(?,?,?,?,?);''',(titles,publication_dates,abstracts,content,urls))
    conn.commit()
    time.sleep(1)

    driver.back()
    time.sleep(1)
    
for y in range(1,9):               
    pages=driver.find_element(By.XPATH,'//*[@id="search"]/div[2]/div/div[4]/div/div[3]').click()
    time.sleep(10) 
    for z in range(1,11):                     
        titles = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[1]/span').text  #標題
        publication_dates =  driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[2]').text   #日期  
        abstracts = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]/div/div[3]').text    #文章摘要 
        elements = driver.find_element(By.XPATH, '//*[@id="search"]/div[2]/div/div[2]/div/div[2]/div/div/div['+str(z)+']/a[2]')
        urls = elements.get_attribute('href')   #連結

        driver.get(urls)           
        a = driver.find_elements(By.XPATH, '/html/body/div[1]/section[4]/section/div/section/div/div/div[1]/div/div[3]/div[5]')
        time.sleep(2)
        for item in a:
            content = item.text   #內文  
        cur.execute('''INSERT INTO videos(title,date,abstract,contents,url) VALUES(?,?,?,?,?);''',(titles,publication_dates,abstracts,content,urls))
        conn.commit()
        time.sleep(1)

        driver.back()
        time.sleep(1)

driver.quit()